### Notebook for Open/Closed label clustering

Model : m2_normed 

m2_normed_model was trained by patches with standardized at mean 0 stdv ~1


`Where is the global Mean and Std
/project2/foster/clouds/data/clouds_laads_preprocessed_2000_2018_band28_29_31/global_mean_std
`

` How to load
modeldir = "/project2/foster/clouds/model/m2_02_global_2000_2018_band28_29_31_normed"  
step = 100000 # DO NOT CHANGE so far  
encoder = load_model(model_dir=modeldir, epoch=step, mtype='encoder')  
`


In [1]:
%reload_ext autoreload
%autoreload 2
import os
import sys
import copy
import glob
import random
import numpy as np
import scipy as sc
import pandas as pd
import tensorflow as tf
from pyhdf.SD import SD, SDC 
from scipy import stats

# visualization
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import patches as mpl_patches

# clusterings
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import SpectralClustering

# TSNE
from sklearn.manifold import TSNE as TSNEs

#### processing functions

In [2]:
def load_model(model_dir, epoch, mtype):
    print(" Load {} at {} epoch".format(mtype, epoch))
    model_def = model_dir+'/'+mtype+'.json'
    model_weight = model_dir+'/'+mtype+'-'+str(epoch)+'.h5'
    with open(model_def, "r") as f:
        model = tf.keras.models.model_from_json(f.read())
    model.load_weights(model_weight)
    return model

In [3]:
def compute_agl(encoder, patches_list,n_cluster = 12):
    encs_list = []
    for i in patches_list:
        encs = encoder.predict(i.reshape(1,128,128,6))
        encs_list += [encs.mean(axis=(1,2))]
    features = np.concatenate(encs_list, axis=0)
    method = AgglomerativeClustering(n_clusters=n_cluster)
    _patches_labels = method.fit(features)
    return _patches_labels.labels_, features

#### Load label data
Maybe Ruby's notebook is helpful?

In [ ]:
### Here, you should write script to load your labeled data

#### Z-score standardization

In [6]:
mean_std_dir = "/project2/foster/clouds/data/clouds_laads_preprocessed_2000_2018_band28_29_31/global_mean_std"

In [7]:
gmean = np.load(os.path.join(mean_std_dir, "m2_02_band28_29_31_gmean.npy"))
gstdv = np.load(os.path.join(mean_std_dir, "m2_02_band28_29_31_gstdv.npy"))

In [ ]:
### Here you normalized by global mena and std

#### Agglomerative clustering

In [ ]:
n_cluster =  # iterate from 2

In [ ]:
patches_list = # list of patches e.g. [  array(128,128,6), ...., array(128,128,6) ] array is numpy array

#### Visualize 

Refer `clouds/src_analysis/cloud_label/visualize_patches.ipynb`  

###### visualization item
- clustering result open and closed 

In [ ]:
def show_all_patches():
    """
    creates a grid of displaying which patches are in which clusters, visualization tool
    
    NOTE: Sometimes there are large gaps between patches, this problem can be solved by playing around with figsize 
    (if there are large horizontal gaps between columns, decrease x value, vice versa for y)
    
    Inputs: 
    None
    
    Outputs:
    None, displayes a grid of patches organized by cluster
    """
    #set rows to the max amount of patches in any of the clusters
    #set cols to the number of clusters (one column for each cluster)
    rows = len(max(list_clusters, key=lambda c: len(c.patches)).patches)
    cols = num_clusters
    
    #create plot
    f, axarr = plt.subplots(rows,cols,figsize=(20,10))

    for c in range(cols):
        axarr[0,c].set_title(str(c), fontsize=18)
        arr = get_open_patches(c) + get_closed_patches(c)

        if len(arr) == 1:
            axarr[0,c].imshow(arr[0])
            axarr[0,c].axis('off')
            for r in range(1, rows):
                plt.delaxes(axarr[r][c])
        else:
            arr_iter = iter(arr)
            for r in range(rows):
                try:
                    axarr[r,c].imshow(next(arr_iter))
                    axarr[r,c].axis('off')
                except:
                    while r < rows:
                        plt.delaxes(axarr[r][c])
                        r += 1
                    break
    plt.savefig(to_save + '%d_patch_pics.png'%(num_clusters))
    plt.show()